<h1>MODEL TRAINER</h1>

In [1]:
#LOAD DEPENDENCIES
import os
import cv2
import time
import pickle
import logging
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.utils import class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras import applications
from tensorflow.keras import Model, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import AveragePooling2D, AlphaDropout, Activation, Add, BatchNormalization, Concatenate, Layer, ReLU, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout

from tensorflow.keras.applications.efficientnet import EfficientNetB0 as trainable_model_a
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as trainable_model_b
from tensorflow.keras.applications.resnet_v2 import ResNet50V2 as trainable_model_c

#PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
#LOAD THE DATA
train_data_dir = "data/train/"
validation_data_dir = "data/validation/"
test_data_dir = "data/test/"
img_rows, img_cols = 224, 224
input_shape = (img_rows,img_cols,3)
model_input = Input(shape=input_shape)
print("Data folders found!")
print("The Input size is set to ", model_input) 

Data folders found!
The Input size is set to  KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [3]:
#Set constants
batch_size = 8
epochs = 25
optimizer = Adam(learning_rate=0.00001)
architecture = 'MFNR_standard_do'

DCNN_A = 'DCNN_A'
DCNN_B = 'DCNN_B'
DCNN_C = 'DCNN_C'

In [4]:
#DATA GENERATORS
#Augment Training Data
train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=45,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1,
                                   fill_mode='constant',
                                   brightness_range=[0.1, 1.0])

val_datagen = ImageDataGenerator(rescale = 1. / 255)

test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        classes=['0_normal', 
                 '1_ulcerative_colitis', 
                 '2_polyps', 
                 '3_esophagitis'])

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        classes=['0_normal', 
                 '1_ulcerative_colitis', 
                 '2_polyps', 
                 '3_esophagitis'])

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        classes=['0_normal', 
                 '1_ulcerative_colitis', 
                 '2_polyps', 
                 '3_esophagitis'])

#CHECK  THE NUMBER OF SAMPLES
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

print("Train samples:", nb_train_samples)
print("Validation samples:", nb_validation_samples)
print("Test samples:", nb_test_samples)

if nb_train_samples == 0:
    print("NO DATA TRAIN FOUND! Please check your train data path and folders!")
else:
    print("Train samples found!")
    
if nb_validation_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Validation samples found!")
    
if nb_test_samples == 0:
    print("NO DATA TEST FOUND! Please check your test data path and folders!")
    print("Check the data folders first!")
else:
    print("Test samples found!")

#check the class indices
train_generator.class_indices
validation_generator.class_indices
test_generator.class_indices

#true labels
Y_test=validation_generator.classes
test_labels = test_generator.classes

num_classes= len(train_generator.class_indices)

print('Model set to train', num_classes, 'classes')

if nb_train_samples and nb_validation_samples and nb_test_samples > 0:
    print("Generators are set!")
    print("Check if dataset is complete and has no problems before proceeding.")

Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
Train samples: 3200
Validation samples: 2000
Test samples: 800
Train samples found!
Validation samples found!
Test samples found!
Model set to train 4 classes
Generators are set!
Check if dataset is complete and has no problems before proceeding.


In [5]:
#Weigh each class

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)
print(class_weights)

[1. 1. 1. 1.]


c:\users\franc\anaconda3\envs\replicate\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1 2 3], y=[0 0 0 ... 3 3 3] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [6]:
# Model
#EfficientNetB0

builder_a = DCNN_A + '_builder'

#TRANSFER LEARNING
def builder_a(model_input):
    builder_a = trainable_model_a(weights='imagenet', 
                                    include_top=False, 
                                    input_tensor = model_input)

#PARTIAL LAYER FREEZING
    for layer in builder_a.layers:
        layer.trainable = False
        
    for layer in builder_a.layers:
        layer._name = layer.name + '_' + DCNN_A
        
    for BatchNormalization in builder_a.layers:
        BatchNormalization.trainable = False

    x = builder_a.layers[-17].output
    
#AUXILIARY FUSING LAYER (AuxFL)    
    x = Conv2D(192, 1, padding='valid', activation='selu', kernel_initializer='lecun_normal')(x)
    x = AveragePooling2D(1, 1)(x)
    x = Dropout(0.2)(x)

    dcnn_a = Model(inputs=builder_a.input, outputs=x, name=DCNN_A)
    return dcnn_a

#INITIALIZE THE MODEL
dcnn_a = builder_a(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")

dcnn_a.summary()

print("successfully built!")

PLEASE CHECK THE ENTIRE MODEL UP TO THE END
Model: "DCNN_A"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_DCNN_A (InputLayer)     [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_DCNN_A (Rescaling)    (None, 224, 224, 3)  0           input_1_DCNN_A[0][0]             
__________________________________________________________________________________________________
normalization_DCNN_A (Normaliza (None, 224, 224, 3)  7           rescaling_DCNN_A[0][0]           
__________________________________________________________________________________________________
stem_conv_pad_DCNN_A (ZeroPaddi (None, 225, 225, 3)  0           normalization_DCNN_A[0][0]       
_________________________________________________

In [7]:
# Model
#MobileNetV2

builder_b = DCNN_B + '_builder'

#TRANSFER LEARNING
def builder_b(model_input):
    builder_b = trainable_model_b(weights='imagenet', 
                                    include_top=False, 
                                    input_tensor = model_input)

#PARTIAL LAYER FREEZING
    for layer in builder_b.layers:
        layer.trainable = False
        
    for layer in builder_b.layers:
        layer._name = layer.name + '_' + DCNN_B
        
    for BatchNormalization in builder_b.layers:
        BatchNormalization.trainable = False
    
    x = builder_b.layers[-39].output

#AUXILIARY FUSING LAYER (AuxFL)
    x = Conv2D(192, 8, padding='valid', activation='selu', kernel_initializer='lecun_normal')(x)
    x = AveragePooling2D(1, 1)(x)
    x = Dropout(0.2)(x)

    dcnn_b = Model(inputs=builder_b.input, outputs=x, name=DCNN_B)
    return dcnn_b

#INITIALIZE THE MODEL
dcnn_b = builder_b(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")

dcnn_b.summary()

print(" successfully built!")

PLEASE CHECK THE ENTIRE MODEL UP TO THE END
Model: "DCNN_B"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_DCNN_A_DCNN_B (InputLay [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_DCNN_B (Conv2D)           (None, 112, 112, 32) 864         input_1_DCNN_A_DCNN_B[0][0]      
__________________________________________________________________________________________________
bn_Conv1_DCNN_B (BatchNormaliza (None, 112, 112, 32) 128         Conv1_DCNN_B[0][0]               
__________________________________________________________________________________________________
Conv1_relu_DCNN_B (ReLU)        (None, 112, 112, 32) 0           bn_Conv1_DCNN_B[0][0]            
_________________________________________________

In [8]:
# Model
#ResNet50V2

builder_c = DCNN_C + '_builder'

#TRANSFER LEARNING
def builder_c(model_input):
    builder_c = trainable_model_c(weights='imagenet', 
                                    include_top=False, 
                                    input_tensor = model_input)

#PARTIAL LAYER FREEZING
    for layer in builder_c.layers:
        layer.trainable = False
        
    for layer in builder_c.layers:
        layer._name = layer.name + '_' + DCNN_C
        
    for BatchNormalization in builder_c.layers:
        BatchNormalization.trainable = False
    
    x = builder_c.layers[-117].output  

#AUXILIARY FUSING LAYER (AuxFL)
    x = Conv2D(192, 6, padding='valid', activation='selu', kernel_initializer='lecun_normal')(x)
    x = AveragePooling2D(3, 3)(x)
    x = Dropout(0.2)(x)

    dcnn_c = Model(inputs=builder_c.input, outputs=x, name=DCNN_C)
    return dcnn_c

#INITIALIZE THE MODEL
dcnn_c = builder_c(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")

dcnn_c.summary()

print("successfully built!")

PLEASE CHECK THE ENTIRE MODEL UP TO THE END
Model: "DCNN_C"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_DCNN_A_DCNN_B_DCNN_C (I [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad_DCNN_C (ZeroPadding2D (None, 230, 230, 3)  0           input_1_DCNN_A_DCNN_B_DCNN_C[0][0
__________________________________________________________________________________________________
conv1_conv_DCNN_C (Conv2D)      (None, 112, 112, 64) 9472        conv1_pad_DCNN_C[0][0]           
__________________________________________________________________________________________________
pool1_pad_DCNN_C (ZeroPadding2D (None, 114, 114, 64) 0           conv1_conv_DCNN_C[0][0]          
_________________________________________________

In [9]:
#RE-INITIALIZE FOR FUSION
dcnn_a = builder_a(model_input)
dcnn_b = builder_b(model_input)
dcnn_c = builder_c(model_input)

print("Accomplished Pre-training and ready for fusion")

Accomplished Pre-training and ready for fusion


In [10]:
#FUSE THE MODELS INTO A SINGLE PIPELINE

models = [dcnn_a, 
          dcnn_b,
          dcnn_c]

print("Fusion success!")
print("Ready to connect with its ending layers!")

Fusion success!
Ready to connect with its ending layers!


In [11]:
#Fuse the models
def mfnr_builder(models, model_input):
    outputs = [m.output for m in models]
    
#INITIAL FUSION LAYER
    y = Add(name='InitialFusionLayer')(outputs)
    
#FINE-TUNING
    y = GlobalAveragePooling2D()(y)
    y = Dropout(0.5)(y)
    prediction = Dense(num_classes,activation='softmax', name='Softmax_Classifier' + architecture)(y)
    model = Model(model_input, prediction, name=architecture)
    return model

#Istantitate the model and report the summary
mfnr = mfnr_builder(models, model_input)

print()
print()
print("PLEASE CHECK THE MODEL UP TO THE END")
print()
print()
print()

mfnr.summary()
print("Complete and ready for compilation and training!")



PLEASE CHECK THE MODEL UP TO THE END



Model: "MFCNN_selu_do"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_DCNN_A_DCNN_B_DCNN_C_DC [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1_DCNN_A (Rescaling)  (None, 224, 224, 3)  0           input_1_DCNN_A_DCNN_B_DCNN_C_DCNN
__________________________________________________________________________________________________
normalization_1_DCNN_A (Normali (None, 224, 224, 3)  7           rescaling_1_DCNN_A[0][0]         
__________________________________________________________________________________________________
stem_conv_pad_DCNN_A (ZeroPaddi (None, 225, 225, 3)  0           normalization_1_DCNN_A[0][0]     
____________________________________________

c:\users\franc\anaconda3\envs\replicate\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [12]:
last_conv_layer_name = mfnr.layers[-4].name

if last_conv_layer_name == 'InitialFusionLayer':
    print("CORRECT LAYER SELECTED:", last_conv_layer_name)
else:
    print("INCORRECT LAYER SELECTED:", last_conv_layer_name)
    print("Please Reselect")

CORRECT LAYER SELECTED: FusionLayer


In [13]:
#MODEL COMPILATION WITH HYPER-PARAMETERS, LOSS FUNCTIONS AND TRAINING!

mfnr.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2,
                              verbose=1, mode='max')

callbacks = [reduce_lr]

# MODEL TRAINING

history = mfnr.fit(train_generator, steps_per_epoch=nb_train_samples // batch_size,
                                  epochs=epochs, validation_data=validation_generator,
                                  callbacks=callbacks, 
                                  validation_steps=nb_validation_samples // batch_size, verbose=1)

start_time = time.time()

elapsed_time = time.time() - start_time
train_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print()
print()
print(train_time, 'train_time')
print()
print(elapsed_time, 'Seconds')
print()
print()
print("MODEL SERIALIZING WAIT FOR A MOMENT...")
print()

#SAVE AFTER TRAINING COMPLETES

mfnr.save('models/' + architecture + '/model.h5')

# #SAVE THE HISTORY FOR EVALUATION

def save_h(file, history):
    with open(file + '/' + architecture + '/model.history', 'wb') as file_pi:
        pickle.dump(history, file_pi)
    print("history saved")

save_h('models/', history.history)

print()
print()
print("The Model weights and history are successfully trained and saved!")

Epoch 1/25
400/400 [==============================] - 51s 113ms/step - loss: 1.2583 - accuracy: 0.6350 - val_loss: 0.2935 - val_accuracy: 0.8645
Epoch 2/25
400/400 [==============================] - 44s 109ms/step - loss: 0.5431 - accuracy: 0.8153 - val_loss: 0.2953 - val_accuracy: 0.8920
Epoch 3/25
400/400 [==============================] - 44s 110ms/step - loss: 0.4230 - accuracy: 0.8581 - val_loss: 0.2916 - val_accuracy: 0.8870
Epoch 4/25
400/400 [==============================] - 44s 111ms/step - loss: 0.3848 - accuracy: 0.8694 - val_loss: 0.1580 - val_accuracy: 0.9295
Epoch 5/25
400/400 [==============================] - 44s 110ms/step - loss: 0.3425 - accuracy: 0.8866 - val_loss: 0.1312 - val_accuracy: 0.9485
Epoch 6/25
400/400 [==============================] - 44s 111ms/step - loss: 0.3208 - accuracy: 0.8944 - val_loss: 0.1398 - val_accuracy: 0.9510
Epoch 7/25
400/400 [==============================] - 44s 110ms/step - loss: 0.3074 - accuracy: 0.8978 - val_loss: 0.1558 - val_ac

In [13]:
#Re-load the trained model

model = load_model('models/' + architecture + '/model.h5')
print("The model is loaded")

The model is loaded


In [14]:
#save json file

def save_m(file, model):
    model.save_weights(file + '/' + architecture + '.h5')
    model_json = model.to_json()
    with open(file + '/' + architecture + '.json', "w") as json_file:
        json_file.write(model_json)
    print("model saved")
    
save_m('models/' + architecture + '/', model)

model saved
